In [1]:
import numpy as np

from astropy.time import Time
from astropy.coordinates import EarthLocation
from datetime import datetime

In [2]:
### Fixed Data

# Observatory data
latitude = -23.21711 
longitude = -45.87021 
height = 620  # Altitude in meters

mu = 398600

In [3]:
### Functions

def convert_to_GCRS(latitude, longitude, height, obs_time):
    # Convert observatory position to GCRS

    # X-axis is in the Earth's equatorial plane but is fixed with the rotation of the Earth so that it passes through the Greenwich meridian (0° longitude).
    # WGS -> GEO
    location = EarthLocation(lat=latitude, lon=longitude, height=height, ellipsoid = 'WGS84')

    # Convert the input datetime to an Astropy Time object
    time = Time(obs_time, location=location)

    # GEO -> GCRS
    gcrs_coords = location.get_gcrs(time)

    return gcrs_coords

def stumpS(z):
    if z > 0:
        s = (np.sqrt(z) - np.sin(np.sqrt(z)))/(np.sqrt(z))**3
    elif z < 0:
        s = (np.sinh(np.sqrt(-z)) - np.sqrt(-z))/(np.sqrt(-z))**3
    else:
        s = 1/6

    return s

def stumpC(z):
    if z > 0:
        c = (1 - np.cos(np.sqrt(z)))/z
    elif z < 0:
        c = (np.cosh(np.sqrt(-z)) - 1)/(-z)
    else:
        c = 1/2

    return c

def kepler_U(dt, r0, vr0, a):
    error = 1e-8
    nMax = 1000

    # Starting value for x
    x = np.sqrt(mu)*np.abs(a)*dt    

    # Iterate until convergence
    n = 0
    ratio = 1

    while np.abs(ratio) > error and n <= nMax:
        n = n + 1
        C = stumpC(a*x**2)
        S = stumpS(a*x**2)
        F = r0*vr0/np.sqrt(mu)*x**2*C + (1 - a*r0)*x**3*S + r0*x - np.sqrt(mu)*dt
        dFdx = r0*vr0/np.sqrt(mu)*x*(1 - a*x**2*S) + (1 - a*r0)*x**2*C + r0
        ratio = F/dFdx
        x = x - ratio
    
    if n > nMax:
        print('No. Iterations of Kepler equation exceeded limit')

    return x


def f_and_g(x, t, r0, a):
    z = a*x**2
    f = 1 - x**2/(r0*stumpC(z))
    g = t - 1/np.sqrt(mu)*x**3*stumpS(z)

    return [f, g]

def coe_from_sv(R, V, mu):
    eps = 1.e-10
    r = np.linalg.norm(R)
    v = np.linalg.norm(V)
    vr = np.dot(R, V) / r
    H = np.cross(R, V)
    h = np.linalg.norm(H)

    incl = np.arccos(H[2] / h)

    N = np.cross([0, 0, 1], H)
    n = np.linalg.norm(N)

    if n != 0:
        RA = np.arccos(N[0] / n)
        if N[1] < 0:
            RA = 2 * np.pi - RA
    else:
        RA = 0

    E = 1 / mu * ((v**2 - mu / r) * R - r * vr * V)
    e = np.linalg.norm(E)

    if n != 0:
        if e > eps:
            w = np.arccos(np.dot(N, E) / n / e)
            if E[2] < 0:
                w = 2 * np.pi - w
        else:
            w = 0
    else:
        w = 0

    if e > eps:
        TA = np.arccos(np.dot(E, R) / e / r)
        if vr < 0:
            TA = 2 * np.pi - TA
    else:
        cp = np.cross(N, R)
        if cp[2] >= 0:
            TA = np.arccos(np.dot(N, R) / n / r)
        else:
            TA = 2 * np.pi - np.arccos(np.dot(N, R) / n / r)

    a = h**2 / mu / (1 - e**2)
    
    coe = [h, e, RA, incl, w, TA, a]
    
    return coe

In [4]:
# Observation Parameters
# Adjusted according to the date and time of the fits file and the exposure time

# Times
t1 = datetime(2023, 10, 25, 19, 32, 21, 0)
t2 = datetime(2023, 10, 25, 19, 32, 22, 500000)
t3 = datetime(2023, 10, 25, 19, 32, 24, 0)

tau1 = (t1 - t2).total_seconds()
tau3 = (t3 - t2).total_seconds()

tau = tau3 - tau1

# RA and Dec
ra1 = 43.527
ra1 = np.radians(ra1)
dec1 = -8.7833
dec1 = np.radians(dec1)

ra2 = 54.420
ra2 = np.radians(ra2)
dec2 = -12.074
dec2 = np.radians(dec2)

ra3 = 64.318
ra3 = np.radians(ra3)
dec3 = -15.105
dec3 = np.radians(dec3)

# Observatory position vector in the specified times 

R1x = convert_to_GCRS(latitude, longitude, height, t1).cartesian.x.value
R1y = convert_to_GCRS(latitude, longitude, height, t1).cartesian.y.value
R1z = convert_to_GCRS(latitude, longitude, height, t1).cartesian.z.value
R1 = np.array([R1x, R1y, R1z])/10**3 # in km
 
R2x = convert_to_GCRS(latitude, longitude, height, t2).cartesian.x.value
R2y = convert_to_GCRS(latitude, longitude, height, t2).cartesian.y.value
R2z = convert_to_GCRS(latitude, longitude, height, t2).cartesian.z.value
R2 = np.array([R2x, R2y, R2z])/10**3 # in km

R3x = convert_to_GCRS(latitude, longitude, height, t3).cartesian.x.value
R3y = convert_to_GCRS(latitude, longitude, height, t3).cartesian.y.value
R3z = convert_to_GCRS(latitude, longitude, height, t3).cartesian.z.value
R3 = np.array([R3x, R3y, R3z])/10**3 # in km

In [5]:
# Auxiliar calculations 

# Finding rho directions IJK

Rho1 = np.array([np.cos(dec1)*np.cos(ra1), np.cos(dec1)*np.sin(ra1), np.sin(dec1)])
Rho2 = np.array([np.cos(dec2)*np.cos(ra2), np.cos(dec2)*np.sin(ra2), np.sin(dec2)])
Rho3 = np.array([np.cos(dec3)*np.cos(ra3), np.cos(dec3)*np.sin(ra3), np.sin(dec3)])


# Auxiliar parameters
p1 = np.cross(Rho2, Rho3)
p2 = np.cross(Rho1, Rho3)
p3 = np.cross(Rho1, Rho2)

D0 = np.dot(Rho1, p1)
D = [[np.dot(R1, p1), np.dot(R1, p2), np.dot(R1, p3)],
      [np.dot(R2, p1), np.dot(R2, p2), np.dot(R2, p3)],
      [np.dot(R3, p1), np.dot(R3, p2), np.dot(R3, p3)]]
E = np.dot(R2, Rho2)

A = 1/D0*(-D[0][1]*tau3/tau + D[1][1] + D[2][1]*tau1/tau)
B = 1/(6*D0)*(D[0][1]*(tau3**2 - tau**2)*tau3/tau + D[2][1]*(tau**2 - tau1**2)*tau1/tau)

a = -(A**2 + 2*A*E + np.linalg.norm(R2)**2)
b = -2*mu*B*(A + E)
c = -(mu*B)**2

In [6]:
# Polynomial equation roots
Roots = np.roots([1, 0, a, 0, 0, b, 0, 0, c])
x = [root.real for root in Roots if root.imag == 0 and root.real > 0][0] # Getting the first positive root

# LaGrange coefficients
f1 = 1 - 1/2*mu*tau1**2/x**3
f3 = 1 - 1/2*mu*tau3**2/x**3

g1 = tau1 - 1/6*mu*(tau1/x)**3
g3 = tau3 - 1/6*mu*(tau3/x)**3

# rho norm

rho2 = A + mu*B/x**3
rho1 = 1/D0*((6*(D[2][0]*tau1/tau3 + D[1][0]*tau/tau3)*x**3 + mu*D[2][0]*(tau**2 - tau1**2)*tau1/tau3)/(6*x**3 + mu*(tau**2 - tau3**2)) - D[0][0])
rho3 = 1/D0*((6*(D[0][2]*tau3/tau1 - D[1][2]*tau/tau1)*x**3 + mu*D[0][2]*(tau**2 - tau3**2)*tau3/tau1)/(6*x**3 + mu*(tau**2 - tau1**2)) - D[2][2])

# Calculating r's and v's in position 2

r1 = R1 + rho1*Rho1
r2 = R2 + rho2*Rho2
r3 = R3 + rho3*Rho3

v2 = (-f3*r1 + f1*r3)/(f1*g3 - f3*g1)

print(r2)
print(v2)

[ 1097.24052972 -5761.93551327 -2501.01182227]
[ 0.57735404 -0.0701011   0.07432516]


In [7]:
# Iterating to improve the results

r_old = r2
v_old = v2

rho1_old = rho1
rho2_old = rho2
rho3_old = rho3

diff1 = 1
diff2 = 1
diff3 = 1

n = 0
nmax = 10000
tol = 1e-8

while diff1 > tol and diff2 > tol and diff3 > tol and n < nmax:
    n = n + 1

    # Kepler Equation
    r0 = np.linalg.norm(r2)
    v0 = np.linalg.norm(v2) 
    vr0 = np.dot(v2, r2)/r0
    a = 2/r0 - v0**2/mu

    x1 = kepler_U(tau1, r0, vr0, a)
    x3 = kepler_U(tau3, r0, vr0, a)

    [ff1, gg1] = f_and_g(x1, tau1, r0, a)
    [ff3, gg3] = f_and_g(x3, tau3, r0, a)

    # Averaging old and new
    f1 = (f1 + ff1)/2
    f3 = (f3 + ff3)/2
    g1 = (g1 + gg1)/2
    g3 = (g3 + gg3)/2

    c1 = g3/(f1*g3 - f3*g1)
    c3 = -g1/(f1*g3 - f3*g1)

    rho1 = 1/D0*(-D[0][0] + 1/c1*D[1][0] - c3/c1*D[2][0])
    rho2 = 1/D0*(-c1*D[0][1] + D[1][1] - c3*D[2][1])
    rho3 = 1/D0*(-c1/c3*D[0][2] + 1/c3*D[1][2] - D[2][2])
    
    r1 = R1 + rho1*Rho1
    r2 = R2 + rho2*Rho2
    r3 = R3 + rho3*Rho3

    v2 = (-f3*r1 + f1*r3)/(f1*g3 - f3*g1)

    diff1 = np.abs(rho1 - rho1_old)
    diff2 = np.abs(rho2 - rho2_old)
    diff3 = np.abs(rho3 - rho3_old)

    rho1_old = rho1
    rho2_old = rho2
    rho3_old = rho3

    if n >= nmax:
        print('Number of iterations exceeds nmax.')    

r = r2
v = v2

In [9]:
print(r)
print(v)

print(np.linalg.norm(r))

[ 1094.14868318 -5766.25734528 -2499.87513847]
[ 1.04924192 -0.52170847  0.3001289 ]
6379.362101401551
